# 2. 搭建环境并实现简单的应用

## 2.1 搭建环境

In [1]:
# curl -fsSL https://ollama.com/install.sh | sh
# ollama pull gpt-oss:20b
# venv/bin/pip3 install langchain==0.1.14 langchain-community==0.0.38 langchain-core==0.1.53 langchain-openai==0.1.7

import langchain
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
langchain.__version__

'0.1.14'

## 2.2 实现问答应用 — 基于基础模型

In [3]:
from langchain_community.llms import Ollama

llm = Ollama(model="gpt-oss:20b")
print(llm.predict("简要说明AI会对人类文明产生深远的影响吗？"))

AI 正在从多个层面重塑人类文明，影响深远且互相交织：

1. **经济与劳动**：自动化与智能化将极大提升生产效率，创造全新职业与产业，但也会导致传统岗位的消失和收入不平等加剧。  
2. **科学与技术突破**：大数据、机器学习与模型驱动的实验可加速药物研发、材料设计与宇宙探索，突破人类认知边界。  
3. **社会与治理**：AI 能优化城市交通、能源分配和公共安全，但也带来隐私泄露、算法偏见与决策透明度等伦理挑战。  
4. **文化与价值观**：生成式 AI 让艺术与创作民主化，同时也引发版权、原创性与人类创意本质的辩论。  
5. **安全与伦理**：从网络安全到军用自主武器，AI 的双刃效应要求建立全球治理机制与可解释性技术。  

总体而言，AI 既是推动人类文明前进的强大引擎，也是引发深刻社会变革与道德重塑的关键因素。


## 2.3 实现翻译应用 — 基于聊天模型和指令模板

In [4]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# 创建一个ChatOllama对象
chat = ChatOllama(model="gpt-oss:20b")
# 配置聊天模板信息
template = "你是一个翻译助理，请将用户输入的内容由{input_language}直接翻译为{output_language}."  # 设置翻译任务的基本指令
system_message_prompt = SystemMessagePromptTemplate.from_template(template)                     # 基于template创建系统消息指令，向聊天模型明确翻译任务要求
human_template = "{text}"                                                                       # 简单定义人类校址指令模板，仅包含待翻译文本
human_massage_prompt = HumanMessagePromptTemplate.from_template(human_template)                 # 定义人类消息指令
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_massage_prompt])   # 将系统消息指令和人类消息指令组合为完整的聊天指令，用于与聊天模型交互

# 输入信息并获取翻译结果
print(chat.invoke(chat_prompt.format_prompt(input_language="英语",
                                            output_language="中文",
                                            text="Artificial Intelligence (AI) will have a profound impact on human civilization in the coming years.").to_messages()))

content='人工智能（AI）将在未来几年对人类文明产生深远影响。' response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-12-30T09:43:45.333648472Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'done_reason': 'stop', 'total_duration': 708226415, 'load_duration': 103208178, 'prompt_eval_count': 222, 'prompt_eval_duration': 5043949, 'eval_count': 18, 'eval_duration': 65090498} id='run-92c89444-bbae-430b-bd3b-b2e523b4303d-0'
